In [1]:
import numpy as np
import pandas as pd
import os
import re
# import matplotlib.pyplot as plt
from PIL import Image


from pathlib import Path
import json
import shutil

os.getcwd()
!pwd



/usr/users/vogg/monkey-tracking-in-the-wild/notebooks


## Converting VoTT output

After labeling with VoTT, we obtain a .csv file, which we convert into many .txt files, one file for every frame. These will be saved in a folder called labels_with_ids (if it does not exist you have to create it.

### Paths to everything

#### Paths for Lemur & Box

In [10]:
data_root = Path("/home/matthias/storage/sfb1528_storage/Z02/LemursAndBoxes/vott-csv-export/")
data_out = Path("/home/matthias/storage/datasets/lemur_box/train/")

#### Paths Richards things

In [2]:
#data_root = Path("/usr/users/vogg/Labelling/Lemurs/LemurLabellingNov22/Training_images_easy/")
data_root = Path("/usr/users/agecker/datasets/MacaqueImagePairs/")

#### Paths Branch Interaction

In [ ]:
data_root = Path("/home/matthias/storage/sfb1528_storage/Z01/MonkeyBranchInteraction/")
data_out = data_root / "train"

#### Paths Exploration Room

In [6]:
data_root = Path("/home/matthias/storage/datasets/explorationroom_largest/validation")
data_out = data_root


In [36]:
labels = pd.read_csv(data_root / "Images_easy-export.csv")

### Test output of labels

In [37]:
# labels = pd.read_csv(data_root + "images/Baboon-finder-export.csv")
# labels = data_root
# labels = pd.read_csv(data_root / 'Lemur_new-export.csv')

In [38]:
labels.head()

,image,xmin,ymin,xmax,ymax,label
0,GX010002_small_3639.jpg,1087.940085,93.059995,1185.829428,216.899973,Lemur
1,GX010002_small_3639.jpg,1378.729169,199.620016,1596.101218,343.620016,Lemur
2,GX010002_small_3639.jpg,617.207123,739.619962,673.349600,798.660049,Box
3,GX010002_small_3639.jpg,798.590419,424.259995,864.809813,490.499973,Box
4,GX010002_small_3639.jpg,1021.720690,154.979984,1119.610144,261.540005,Box


### Fixing paths to image and label files

In [39]:
# img_list = os.listdir(data_root + "images")
imgs = data_root
# lbls = data_root / 'labels_with_ids'
# imgs = data_root
#lbls = data_root / 'VoTT_output'
# img_list = imgs.glob('**/*.png')
img_list = os.listdir(data_root / "images")
img_list = [img for img in img_list if not img.startswith(".")]
#labels_list = lbls.glob('**/*.json')
#regex = re.compile('.*jpg')
#img_list = sorted([i for i in img_list if regex.match(i)])

In [40]:
len(img_list)

500

### Get all resolutions of images

In [8]:
sh = []
for img in img_list:
    img=np.asarray(Image.open(img))
    sh.append((img.shape))
# print(sh)
print(np.unique(sh))

# ken front left camera has other resolution than the rest: 768x768 instead of 768x1024

FileNotFoundError: [Errno 2] No such file or directory: 'a_e_9_221007_c2_5151.jpg'

### Copy images to train location

In [12]:
# save all pngs to images
for img in img_list:
    shutil.copy(img, (data_root.parent.parent / 'train' / 'images' / img.name))

    # sort out imgs without corresponding labeling
    # tmp = (lbls / img.name).with_suffix('.txt')
    # if not tmp.is_file():
    #     img.rename((data_root / 'sorted_out' / img.name))


### Get image shapes

In [17]:
# img_id = img_list[1]
img_path = next(img_list)
# img = np.asarray(Image.open(data_root + "images/" + img_id))
img = np.asarray(Image.open(img_path))
#fig, ax = plt.subplots(figsize=(640/96, 640/96), dpi=96)
#ax.imshow(img)
img_h, img_w, _ = img.shape

TypeError: 'list' object is not an iterator

# Extracting multiple json VoTT output for single-class FairMOT

- used for testing Zurnas Explorationroom labeling with single class FairMOT 
- all classes labeled in VoTT but only monkey class BB's are extracted

In [ ]:
# cell to extract monkey class
i_monkey = 0
for label in labels_list:
    # print(label)
    with open(label) as f:
        content = json.load(f)
        
    origin_path = Path(content["asset"]["path"])
    image_path_end = Path(origin_path.parent.name, origin_path.name)
    image_path = data_root / label.parent.parent.name  / image_path_end
    # print(image_path)
    # img = np.asarray(Image.open(image_path))
    # img_h, img_w, _ = img.shape
    img_h, img_w = content["asset"]["size"].values()

    all_boxes = content["regions"]
    # print(all_boxes[0])
    monkey_boxes = [box for box in all_boxes if "monkey" in box["tags"]]
    # if monkey_boxes:
    #     shutil.copy(image_path, (data_root / 'images' /image_path.name))
    # print(monkey_boxes)

    label_fpath = (data_root / "labels_with_ids" / image_path.name).with_suffix('.txt')
    # print(label_fpath)

    for monkey_box in monkey_boxes:
        id = monkey_box["id"]
        h, w, left, top = monkey_box["boundingBox"].values()
        # print(h,w,left,top)

        x_center = left + w/2
        y_center = top + h/2

        #  img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_monkey += 1
        with label_fpath.open(mode='a') as f:
            f.write(label_str)
    # plotBBox(str(image_path.stem),str(image_path.parent)+'/',str(label_fpath.parent)+'/', ending='.png')
        


# Extracting multiple json VoTT output for multi-class FairMOT

- used for Zurnas Explorationroom labeling

In [68]:
# cell for all classes
i_obj = [0,0,0,0,0]

for label in labels_list:
    # print(label)
    with open(label) as f:
        content = json.load(f)
        
    origin_path = Path(content["asset"]["path"])
    image_path_end = Path(origin_path.parent.name, origin_path.name)
    image_path = data_root / label.parent.parent.name  / image_path_end
    # print(image_path)
    # img = np.asarray(Image.open(image_path))
    # img_h, img_w, _ = img.shape
    img_h, img_w = content["asset"]["size"].values()

    all_boxes = content["regions"]

    # monkey_boxes = [box for box in all_boxes if "monkey" in box["tags"]]

    label_fpath = (data_root / "labels_with_ids" / image_path.name).with_suffix('.txt')

    for box in all_boxes:
        if "monkey" in box["tags"]:
            obj_class = 0
        elif "patch" in box["tags"]:
            obj_class = 1
        elif "kong" in box["tags"]:
            obj_class = 2
        elif "branch" in box["tags"]:
            obj_class = 3
        elif "XBI" in box["tags"]:
            obj_class = 4
        h, w, left, top = box["boundingBox"].values()
        # print(h,w,left,top)

        x_center = left + w/2
        y_center = top + h/2

        #  img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_obj[obj_class] += 1
        with label_fpath.open(mode='a') as f:
            f.write(label_str)

# Extracting one csv VoTT output for multi-class FairMOT

- used for lemurs and boxes

In [41]:
i_obj = [0,0]
img_paths = []
for _, row in labels.iterrows():
    # print(label)
    image, xmin, ymin, xmax, ymax, label = row

    image_path = data_root / "images" / image
    img_paths.append(str(image_path))
    # print(image_path)
    img = np.asarray(Image.open(image_path))
    img_h, img_w, _ = img.shape

    # for lemur and box ---
    # img_h, img_w = [2160, 3840]


    # label_fpath = (data_out / 'labels' / image_path.name).with_suffix('.txt')
    # for exp room
    if not os.path.exists(data_root / 'labels_with_ids' ):
        os.makedirs(data_root / 'labels_with_ids' )

    label_fpath = (data_root / 'labels_with_ids' / image_path.name).with_suffix('.txt')
    
    obj_class = 0

    # for lemur and box ---
    # if "Lemur" in label:
    #     obj_class = 0
    # elif "Box" in label:
    #     obj_class = 1

    w = (xmax - xmin)
    h = (ymax - ymin)

    x_center = xmin + w / 2
    y_center = ymin + h / 2

    #  img_id = re.sub("[.]jpg","", img_id)
    #Label-String schreiben.

    label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
            obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, w / img_w, h / img_h)

    i_obj[obj_class] += 1

    # not for lemur box
    with label_fpath.open(mode='a') as f:
        f.write(label_str)

# for lemur box
# img_paths = list(set(img_paths))
# with (data_out / 'lemur_box.train').open('w') as f:
#     for ip in img_paths:
#         f.write(f'{ip}\n')

# Extracting one csv VoTT output for multi-class FairMOT + classification

In [11]:
dict_indivs = {'Cha': 0, 'Flo': 1, 'Gen': 2, 'Geo': 3, 'Her': 4, 'Rab': 5, 'Red': 6,
              'Uns': 7}

In [14]:
i_obj = [0,0]
img_paths = []
for _, row in labels.iterrows():
    # print(label)
    image, xmin, ymin, xmax, ymax, label = row
    #indiv = label.split("-")[1][:3]
    indiv = label[:3]

    image_path = data_root / 'eval' / image
    img_paths.append(str(image_path))
    # print(image_path)
    img = np.asarray(Image.open(image_path))
    img_h, img_w, _ = img.shape

    # for lemur and box ---
    # img_h, img_w = [2160, 3840]


    # label_fpath = (data_out / 'labels' / image_path.name).with_suffix('.txt')
    # for exp room
    label_fpath = (data_root / 'labels_with_ids' / image_path.name).with_suffix('.txt')
    
    obj_class = 0

    # for lemur and box ---
    if "Lemur" in label:
        obj_class = 0
    elif "Box" in label:
        obj_class = 1

    w = (xmax - xmin)
    h = (ymax - ymin)

    x_center = xmin + w / 2
    y_center = ymin + h / 2

    #  img_id = re.sub("[.]jpg","", img_id)
    #Label-String schreiben.

    label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f} {:d}\n'.format(
            obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, 
        w / img_w, h / img_h, dict_indivs[indiv])

    i_obj[obj_class] += 1

    # not for lemur box
    with label_fpath.open(mode='a') as f:
        f.write(label_str)
    



# Original VoTT output transform for FairMOT (from Richard)

- I at least think that that is in this cell ^^

In [332]:
i_monkey = 0

# for img_id in img_list:
for img_path in img_list:
    idx = np.where(labels.image == img_id)
    image_labels = labels.iloc[idx]
    
    label_fpath = data_root + "labels_with_ids/" + re.sub(".JPG", ".txt", img_id)

    img = np.asarray(Image.open(data_root + "images/" + img_id))
    img_h, img_w, _ = img.shape

    for index, row in image_labels.iterrows():

        #if row.label == "baboon":
        x_center = row.xmin + (row.xmax - row.xmin)/2
        y_center = row.ymin + (row.ymax - row.ymin)/2
        w = row.xmax - row.xmin
        h = row.ymax - row.ymin

        #i_monkey = row.label

        img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_monkey += 1
        with open(label_fpath, 'a') as f:
            f.write(label_str)


            

### Converting CVAT output

Data comes in separate files.

In [353]:
video_id = 'VID_20210227_133440_0'

data_root = "/usr/users/agecker/datasets/macaque_videos/"

path_o = data_root + video_id + "/obj_train_data/"

frame_list = os.listdir(path_o)
frame_list.sort()

In [354]:
regex = re.compile('.*txt')
frame_list = [i for i in frame_list if regex.match(i)]
print(len(frame_list))

361


In [343]:

!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/

mkdir: cannot create directory ‘/usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/’: File exists


In [356]:
!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210227_133440_0/images/

In [357]:
for frame_id in frame_list:
    text = pd.read_csv(path_o + frame_id, names = [1,2,3,4,5], sep = " ")
    text.insert(0, "z", np.zeros(len(text)))
    text.z = text.z.astype(int)
    text.to_csv(data_root + video_id + "/labels_with_ids/" + frame_id,
                 sep=' ', index=False, header = False)

## Videos to frames

In [358]:
import cv2
import sys
#sys.path

In [359]:
path = "/usr/users/agecker/datasets/macaque_videos/"
cap = cv2.VideoCapture(path + "Videos/" + video_id + ".mp4")

#Total number of frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)

count = 0

Frame count: 361


In [360]:
#%%

while count < frame_count:
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
    success, image = cap.read()
    cv2.imwrite(path + video_id + "/images/frame_%s.jpg" % str(count).zfill(6), image)     # save frame as JPEG file
    count += 1
    if count % 50 == 0:
        print(count)

50
100
150
200
250
300
350


## Make files for training list images

In [42]:
len(img_list)

500

In [3]:
#can be deleted
folders = os.listdir(data_root)
folders = sorted([folder for folder in folders if folder.endswith("100")])

In [26]:
for folder in folders: 
    img_list = os.listdir(data_root / folder / "labels_with_ids")
    img_list = [img for img in img_list if not img.startswith(".")]
    for img_id in img_list:
        if not img_id.startswith("."):
            label_fpath = data_root / "macaque_image_pairs.train"

            label_str = data_root.name + "/" + folder + "/images/" + img_id.replace("txt", "jpg") + "\n"

            with open(label_fpath, 'a') as f:
                f.write(label_str)

In [11]:
folders

['0_100',
 '1_100',
 '2_100',
 '3_100',
 '4_100',
 '5_100',
 '6_100',
 '7_100',
 '8_100',
 '9_100']

In [10]:
#can be deleted: remove duplicates

i = 1
for folder in folders:
    label_list = os.listdir(data_root / folder / "labels_with_ids")
    label_list = [label for label in label_list if not label.startswith(".")]
    for label_id in label_list:
        with open(data_root / folder / "labels_with_ids" / label_id, "r") as f:
            boxes = f.readlines()
            #boxes = set(boxes)
            #boxes = list(boxes)
            with open(data_root / folder / "labels_with_ids" / label_id, "w") as f:
                for box in boxes:
                    i += 1
                    box = box.split(" ")
                    box[1] = str(i)
                    box = " ".join(box)
                    f.write(box)

In [9]:
for folder in folders:
    label_list = os.listdir(data_root / folder / "labels_with_ids")
    label_list = [label for label in label_list if not label.startswith(".")]
    for label_id in label_list:

        with open(data_root / folder / "labels_with_ids" / label_id, 'r') as file:
            lines = file.readlines()

            # Remove empty rows from the lines
            processed_lines = [line.strip() for line in lines if line.strip()]

            # Write the modified lines to the output file
            with open(data_root / folder / "labels_with_ids" / label_id, 'w') as file:
                file.write('\n'.join(processed_lines))

In [13]:
for folder in folders:
    label_list = os.listdir(data_root / folder / "labels_with_ids")
    label_list = [label for label in label_list if not label.startswith(".")]
    for label_id in label_list:
        if os.path.getsize(data_root / folder / "labels_with_ids" / label_id) == 0:
            print(f"Empty file: {data_root / folder / 'labels_with_ids' / label_id}")

In [15]:
with open(data_root / "macaque_image_pairs.train", 'r') as file:
    lines = file.readlines()

# Remove lines containing the string "_1.jpg"
filtered_lines = [line for line in lines if "_1.jpg" not in line]

# Write the filtered lines to the output file
with open(data_root / "macaque_image_pairs2.train", 'w') as file:
    file.writelines(filtered_lines)


In [43]:
for img_id in img_list:
    if not img_id.startswith("."):
        label_fpath = data_root / "lemur_ids.train"

        label_str = data_root.name + "/images/" + img_id + "\n"

        with open(label_fpath, 'a') as f:
            f.write(label_str)

In [ ]:
# my verion
for i, img in enumerate(img_list):
    train_labl_fpath = '/media/gwdg_home/Data/Monkey/02_VoTT/lab.train'
    val_labl_fpath = '/media/gwdg_home/Data/Monkey/02_VoTT/lab.val'
    if i % 10 == 0:
        with open(val_labl_fpath, 'a') as f:
            f.write(str(img) + '\n')
    else:
        with open(train_labl_fpath, 'a') as f:
            f.write(str(img) + '\n')
